In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
    # so it does not look messy in the lesson

# Wrangling
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Statistical Tests
import scipy.stats as stats

# Visualizing
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import learning_curve

pd.options.display.float_format = '{:20,.2f}'.format
    # set some formats f default number type (2 decimals)

import env
import wrangle
import explore

SyntaxError: invalid syntax (explore.py, line 92)

### Acquire

In [ ]:
df = pd.read_csv("zillow.csv")

In [ ]:
df.info()

### Prepare

In [ ]:
df = wrangle.clean_zillow(df)

In [ ]:
df.info()

In [ ]:
train, validate, test = wrangle.split_zillow(df)

In [ ]:
scale_columns = ['bathrooms', 'bedrooms', 'quality', 
                  'square_feet', 'full_bathrooms', 
                  'latitude', 'longitude', 'lot_square_feet', 'has_pool', 
                  'land_type', 'city', 'county', 'zip_code', 
                 'room_count', 'unitcnt', 'yearbuilt', 'structure_tax_value', 
                 'tax_value',  'land_tax_value', 'taxamount',
                  'house_age', 'tax_rate']

In [ ]:
train_scaled, validate_scaled, test_scaled = wrangle.scale_my_data(train, validate, test, scale_columns)

In [ ]:
train.head()

In [ ]:
binary_vars = ['in_los_angeles', 'in_orange_county', 'in_ventura', 
               'has_heating_system', 'has_fireplace', 'has_pool']
categorical_vars = ['quality', 'full_bathrooms', 'land_type', 'county', 
                    'zip_code']
quant_vars = ['bedrooms', 'bathrooms', 'square_feet', 'lot_square_feet', 
             'tax_rate', 'structure_tax_value', 'tax_value', 
              'land_tax_value', 'taxamount']
categorical_target = 'level_of_log_error'
continuous_target = 'logerror'

In [ ]:
df.level_of_log_error.value_counts()

# Univariate Exploration

In [ ]:
print(f'Boxplots and Histograms of Categoriacal Variables', categorical_vars)
plt.subplots(5, 2, figsize=(20,25), sharey=True)
sns.set(style="darkgrid")

plt.subplot(5,2,1)
plt.title("Distribution of Home Quality", size=10, color='black')
sns.boxplot(train.quality, color='darkred')

plt.subplot(5,2,2)
plt.title("Distribution of Home Quality", size=10, color='black')
sns.histplot(train.quality, color='darkred', ec='black')

plt.subplot(5,2,3)
plt.title("Distribution of Homes Number of Full Bathrooms", size=10, color='black')
sns.boxplot(train.full_bathrooms, color='darkred')

plt.subplot(5,2,4)
plt.title("Distribution of Homes Number of Full Bathrooms", size=10, color='black')
sns.histplot(train.full_bathrooms, color='darkred', ec='black')

plt.subplot(5,2,5)
plt.title("Distribution of Land Type", size=10, color='black')
sns.boxplot(train.land_type, color='darkred')

plt.subplot(5,2,6)
plt.title("Distribution of Land Type", size=10, color='black')
sns.histplot(train.land_type, color='darkred', ec='black')

plt.subplot(5,2,7)
plt.title("Distribution of County", size=10, color='black')
sns.boxplot(train.county, color='darkred')

plt.subplot(5,2,8)
plt.title("Distribution of County", size=10, color='black')
sns.histplot(train.county, color='darkred', ec='black')

plt.subplot(5,2,9)
plt.title("Distribution of Zip Code", size=10, color='black')
sns.boxplot(train.zip_code, color='darkred')

plt.subplot(5,2,10)
plt.title("Distribution of Zip Code", size=10, color='black')
sns.histplot(train.zip_code, color='darkred', ec='black')

In [ ]:
print(f'Boxplots and Histograms of Quantitative Variables', quant_vars)
plt.subplots(8, 2, figsize=(30,50), sharey=True)
sns.set(style="darkgrid")

plt.subplot(8,2,1)
plt.title("Distribution of Bedrooms", size=10, color='black')
sns.boxplot(train.bedrooms, color='darkred')

plt.subplot(8,2,2)
plt.title("Distribution of Bedrooms", size=10, color='black')
sns.histplot(train.bedrooms, color='darkred', ec='black')

plt.subplot(8,2,3)
plt.title("Distribution of Bathrooms", size=10, color='black')
sns.boxplot(train.bathrooms, color='darkred')

plt.subplot(8,2,4)
plt.title("Distribution of Bathrooms", size=10, color='black')
sns.histplot(train.bathrooms, color='darkred', ec='black')

plt.subplot(8,2,5)
plt.title("Distribution of Square Feet of Home", size=10, color='black')
sns.boxplot(train.square_feet, color='darkred')

plt.subplot(8,2,6)
plt.title("Distribution of Square Feet of Home", size=10, color='black')
sns.histplot(train.square_feet, color='darkred', ec='black')

plt.subplot(8,2,7)
plt.title("Distribution of Square Feet of Lot", size=10, color='black')
sns.boxplot(train.lot_square_feet, color='darkred')

plt.subplot(8,2,8)
plt.title("Distribution of Square Feet of Lot", size=10, color='black')
sns.histplot(train.lot_square_feet, color='darkred', ec='black')

plt.subplot(8,2,9)
plt.title("Distribution of Tax Rate", size=10, color='black')
sns.boxplot(train.tax_rate, color='darkred')

plt.subplot(8,2,10)
plt.title("Distribution of Tax Rate", size=20, color='black')
sns.histplot(train.tax_rate, color='darkred', ec='black')

plt.subplot(8,2,11)
plt.title("Distribution of Structure Tax Value", size=20, color='black')
sns.boxplot(train.structure_tax_value, color='darkred')

plt.subplot(8,2,12)
plt.title("Distribution of Structure Tax Value", size=20, color='black')
sns.histplot(train.structure_tax_value, color='darkred', ec='black')

plt.subplot(8,2,13)
plt.title("Distribution of Land Tax Value", size=20, color='black')
sns.boxplot(train.land_tax_value, color='darkred')

plt.subplot(8,2,14)
plt.title("Distribution of Land Tax Value", size=20, color='black')
sns.histplot(train.land_tax_value, color='darkred', ec='black')

plt.subplot(8,2,15)
plt.title("Distribution of Tax Amount", size=20, color='black')
sns.boxplot(train.taxamount, color='darkred')

plt.subplot(8,2,16)
plt.title("Distribution of Tax Amount", size=20, color='black')
sns.histplot(train.taxamount, color='darkred', ec='black')


Takeaways:
- all of them have outliers
    - should run IQR to see about handling these
- pretty much all of them are skewed right

# Bivariate Exploration

In [ ]:
plt.subplots(14,1, figsize=(15,90), sharey=True)
sns.set(style="darkgrid")

plt.subplot(14,1,1)
sns.scatterplot(x='logerror', y='yearbuilt',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and Year the Home was Built?")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    
plt.subplot(14,1,2)
sns.scatterplot(x='logerror', y='quality',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and the Number of Bedrooms?")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(14,1,3)
sns.scatterplot(x='logerror', y='square_feet',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and the Square Footage?")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(14,1,4)
sns.scatterplot(x='logerror', y='lot_square_feet',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and the Square Footage?")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(14,1,5)
sns.scatterplot(x='logerror', y='bedrooms',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and the Square Footage?")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(14,1,6)
sns.scatterplot(x='logerror', y='bathrooms',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and the Square Footage?")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(14,1,7)
sns.scatterplot(x='logerror', y='tax_rate',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and the Square Footage?")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(14,1,8)
sns.scatterplot(x='logerror', y='structure_tax_value',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and the Square Footage?")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(14,1,9)
sns.scatterplot(x='logerror', y='land_tax_value',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and the Square Footage?")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(14,1,10)
sns.scatterplot(x='logerror', y='taxamount',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and the Square Footage?")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(14,1,11)
sns.scatterplot(x='logerror', y='full_bathrooms',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and the Square Footage?")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(14,1,12)
sns.scatterplot(x='logerror', y='land_type',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and the Square Footage?")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(14,1,13)
sns.scatterplot(x='logerror', y='county',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and the Square Footage?")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(14,1,14)
sns.scatterplot(x='logerror', y='zip_code',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and the Square Footage?")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

In [ ]:
plt.figure(figsize=(16,12))
sns.scatterplot(x='logerror', y='square_feet',
               data=train, hue='fips', palette='crest')
plt.title("How is Logerror Affected by Area and the Square Footage?")
plt.show()

# Multivariate Exploration
Standard:

for var in binary_vars:
    explore.explore_multivariate(train, categorical_target, var, quant_vars)

Takeaways:

# Stats

explore.run_stats_on_everything(train, categorical_target, continuous_target, binary_vars, quant_vars)

1. Ask at least 5 questions about the data, keeping in mind that your target variable is logerror. e.g. Is logerror significantly different for properties in LA County vs Orange County vs Ventura County?

2. Answer those questions through a mix of statistical tests and visualizations.

Bonus:
- Compute the mean(logerror) by zipcode and the overall mean(logerror). Write a loop that will run a t-test between the overall mean and the mean for each zip code. We want to identify the zip codes where the error is significantly higher or lower than the expected error.